В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import optuna

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV

%load_ext autoreload
%autoreload all

from process_bank_churn_v2 import preprocess_data, compute_auroc
from process_bank_churn_v2 import preprocess_new_data

## 0.
Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [2]:
data_dir = './data/bank-customer-churn-prediction/'
raw_train_df = pd.read_csv(data_dir + 'train.csv', index_col=0)

In [3]:
data_dict = preprocess_data(raw_train_df,
                            target_col='Exited',
                            scaler_numeric=True)

In [4]:
data_dict.keys()

dict_keys(['train_X', 'train_y', 'val_X', 'val_y', 'scaler', 'encoder', 'input_cols', 'numeric_cols', 'categorical_cols', 'poly_cols', 'poly_transformer'])

## 1. KNeighborsClassifier()
Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [5]:
knn = KNeighborsClassifier()
knn.fit(data_dict['train_X'], data_dict['train_y'])

train_auroc, val_auroc = compute_auroc(knn, data_dict)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

AUROC on Train: 0.955
AUROC on Validation: 0.851


Our model `KNeighborsClassifier` with default parameters tends to overfit the data (has high variance).

## 2. `GridSearchCV` for `KNeighborsClassifier`
Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Встановіть крос валідацію на 5 фолдів.

Після успішного завершення пошуку оптимального гіперпараметру:
- виведіть найкраще значення параметра
- збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
- оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
- зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [30]:
%%time
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 25)}

knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(data_dict['train_X'], data_dict['train_y'])

knn_best = knn_gs.best_estimator_

knn_gs.best_params_

CPU times: total: 33.1 s
Wall time: 33.3 s


{'n_neighbors': np.int64(7)}

In [7]:
train_auroc, val_auroc = compute_auroc(knn_best, data_dict)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

AUROC on Train: 0.947
AUROC on Validation: 0.863


The best value of `n_neighbors` is `7`. However, the `KNeighborsClassifier` still tends to overfit the data. It performs better than the model with the default `n_neighbors = 5`, but worse than `DecisionTreeClassifier(max_depth=5, random_state=24)`.

## 3. `GridSearchCV` for `DecisionTreeClassifier`
Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
- `max_depth` від 1 до 20 з кроком 2
- `max_leaf_nodes` від 2 до 10 з кроком 1

Обовʼязково при цьому ініціюйте модель з фіксацією `random_seed`.

Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

Після успішного завершення пошуку оптимальних гіперпараметрів:
- виведіть найкращі значення параметра
- збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
- оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
- зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [8]:
%%time
dt_model = DecisionTreeClassifier(random_state=24)

params_dt = {'max_depth': np.arange(1, 21, 2),
             'max_leaf_nodes': np.arange(2, 11, 1)}

dt_gs = GridSearchCV(dt_model, params_dt, cv=3, scoring='roc_auc')
dt_gs.fit(data_dict['train_X'], data_dict['train_y'])

dt_best = dt_gs.best_estimator_

dt_gs.best_params_

CPU times: total: 3.81 s
Wall time: 3.83 s


{'max_depth': np.int64(5), 'max_leaf_nodes': np.int64(10)}

In [9]:
train_auroc, val_auroc = compute_auroc(dt_best, data_dict)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

AUROC on Train: 0.903
AUROC on Validation: 0.896


The `DecisionTreeClassifier` with the best parameters `max_depth = 5` and `max_leaf_nodes = 10` still tends to overfit the data. It performs better than the `KNeighborsClassifier` with `n_neighbors = 7`, but worse than the `DecisionTreeClassifier` with handle optimized parameters `max_depth = 5` and `max_leaf_nodes = 25`, which achieved an `AUROC on Validation = 0.919`.

## 4. `RandomizedSearchCV` for `DecisionTreeClassifier`
Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

Після успішного завершення пошуку оптимальних гіперпараметрів
- виведіть найкращі значення параметра
- збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
- оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
- зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
- проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [10]:
%%time
dt_model = DecisionTreeClassifier(random_state=24)

params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

dt_rgs = RandomizedSearchCV(dt_model, params_dt, n_iter=40, cv=3, scoring='roc_auc')
dt_rgs.fit(data_dict['train_X'], data_dict['train_y'])

dt_random_search_best = dt_rgs.best_estimator_
dt_rgs.best_params_

CPU times: total: 1.25 s
Wall time: 1.25 s


{'splitter': 'best',
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_leaf_nodes': np.int64(19),
 'max_features': None,
 'max_depth': np.int64(19),
 'criterion': 'gini'}

In [11]:
train_auroc, val_auroc = compute_auroc(dt_random_search_best, data_dict)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

AUROC on Train: 0.918
AUROC on Validation: 0.910


## 5. 
Якщо у Вас вийшла краща метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

Let's expand the space for `max_leaf_nodes` to `60`.

In [12]:
%%time
dt_model = DecisionTreeClassifier(random_state=24)

params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 11),
    'max_leaf_nodes': np.arange(2, 60),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

dt_rgs = RandomizedSearchCV(dt_model, params_dt, n_iter=200, cv=3, scoring='roc_auc')
dt_rgs.fit(data_dict['train_X'], data_dict['train_y'])

CPU times: total: 5.62 s
Wall time: 5.63 s


RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=24),
                   n_iter=200,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59]),
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   scoring='roc_auc')

In [13]:
train_auroc, val_auroc = compute_auroc(dt_rgs.best_estimator_, data_dict)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

AUROC on Train: 0.933
AUROC on Validation: 0.917


This model perform better, so we could submitting it to Kaggle.

In [14]:
test_df = pd.read_csv(data_dir + 'test.csv')
submission = pd.read_csv(data_dir + 'sample_submission.csv')

test_inputs = preprocess_new_data(
    test_df,
    data_dict['input_cols'],
    data_dict['numeric_cols'],
    data_dict['categorical_cols'],
    data_dict['poly_cols'],
    data_dict['encoder'],
    data_dict['scaler'],
    data_dict['poly_transformer']
)

In [15]:
model = dt_rgs.best_estimator_
model.fit(data_dict['train_X'], data_dict['train_y'])

test_preds_proba = model.predict_proba(test_inputs)[:, 1]

submission['Exited'] = test_preds_proba

submission.to_csv(data_dir + 'submission_dt_rgs.csv',
                  index=False)

<img src='https://i.imgur.com/vRVeQ80.png' width="600">


### degree = 2

Let's try to use `PolynomialFeatures` with `degree = 2`

In [16]:
data_dict_poly_2 = preprocess_data(raw_train_df,
                                   target_col='Exited',
                                   scaler_numeric=True,
                                   polynomial_features=True,
                                   polynomial_degree=2)

In [17]:
%%time
dt_poly_2_model = DecisionTreeClassifier(random_state=24)

params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 11),
    'max_leaf_nodes': np.arange(10, 100),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

dt_poly_2_rgs = RandomizedSearchCV(
    dt_poly_2_model,
    params_dt,
    n_iter=200,
    cv=3,
    scoring='roc_auc')

dt_poly_2_rgs.fit(data_dict_poly_2['train_X'], data_dict_poly_2['train_y'])

print(dt_poly_2_rgs.best_params_)

train_auroc, val_auroc = compute_auroc(dt_poly_2_rgs.best_estimator_, data_dict_poly_2)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

{'splitter': 'best', 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_leaf_nodes': np.int64(32), 'max_features': None, 'max_depth': np.int64(6), 'criterion': 'entropy'}
AUROC on Train: 0.932
AUROC on Validation: 0.917
CPU times: total: 20.9 s
Wall time: 21 s


`DesicionTreeClassifier` with `PolynomialFeature` of `degree = 2` perform worse than without `PolynomialFeature`.

## Scikit-optimize for DesicionTreeClassifier with PolynomialFeature

In [19]:
input_cols = list(raw_train_df.columns)[2:-1]
target_col = 'Exited'

In [20]:
train_df, val_df = train_test_split(
        raw_train_df, test_size=0.2, random_state=24, stratify=raw_train_df[target_col]
    )

train_inputs, train_targets = train_df[input_cols], train_df[target_col]
val_inputs, val_targets = val_df[input_cols], val_df[target_col]

In [ ]:
numeric_cols = train_inputs.select_dtypes('number').columns.to_list()
categorical_cols = train_inputs.select_dtypes('object').columns.to_list()

poly_features = Pipeline(steps=[
    ('poly_features', PolynomialFeatures())
])

categorical_transformer = Pipeline(steps=[
    ('onehot_enc', OneHotEncoder(drop='if_binary',
                                 sparse_output=False,
                                 handle_unknown='ignore'))
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('poly_features', poly_features, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
], remainder='passthrough')

preprocessor.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('poly_features',
                                 Pipeline(steps=[('poly_features',
                                                  PolynomialFeatures())]),
                                 ['CreditScore', 'Age', 'Tenure', 'Balance',
                                  'NumOfProducts', 'HasCrCard',
                                  'IsActiveMember', 'EstimatedSalary']),
                                ('cat',
                                 Pipeline(steps=[('onehot_enc',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Geography', 'Gender'])])

In [22]:
def get_auroc(model, train_inputs, train_targets, val_inputs, val_targets):
    predict_train_y = model.predict_proba(train_inputs)[:, 1]
    predict_val_y = model.predict_proba(val_inputs)[:, 1]

    train_auroc = roc_auc_score(train_targets, predict_train_y)
    val_auroc = roc_auc_score(val_targets, predict_val_y)

    return train_auroc, val_auroc

In [23]:
%%time
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=24))
])

opt = BayesSearchCV(
    model_pipeline,
    {
        'preprocessor__poly_features__poly_features__degree': (1, 6, 'uniform'),
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__splitter': ['best', 'random'],
        'classifier__max_depth': (1, 11, 'uniform'),
        'classifier__max_leaf_nodes': (10, 250, 'uniform'),
        'classifier__min_samples_split': (2, 20, 'uniform'),
        'classifier__min_samples_leaf': (1, 2, 4, 8),
        'classifier__max_features': [None, 'sqrt', 'log2'],

    },
    n_iter=50,
    scoring='roc_auc',
    cv=3,
    random_state=24
)

opt.fit(train_inputs, train_targets)

opt.best_score_, opt.best_params_

CPU times: total: 5min 23s
Wall time: 4min 27s


(np.float64(0.9193857442445014),
 OrderedDict([('classifier__criterion', 'gini'),
              ('classifier__max_depth', 5),
              ('classifier__max_features', None),
              ('classifier__max_leaf_nodes', 250),
              ('classifier__min_samples_leaf', 4),
              ('classifier__min_samples_split', 12),
              ('classifier__splitter', 'best'),
              ('preprocessor__poly_features__poly_features__degree', 1)]))

In [ ]:
opt.best_estimator_.fit(train_inputs, train_targets)
train_auroc, val_auroc = get_auroc(opt.best_estimator_,
                                   train_inputs,
                                   train_targets,
                                   val_inputs,
                                   val_targets)
print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

AUROC on Train: 0.927
AUROC on Validation: 0.919


`BayesSearchCV` from `Scikit-optimize` tuned the model to perform slightly better than `RandomizedSearchCV`.

## Optuna for LogisticRegression with PolynomialFeature

In [32]:
%%time
def objective(trial):

    params = {
        'class_weight': 'balanced',
        'max_iter': 500,
        'random_state': 24,
        'solver': trial.suggest_categorical('solver',
                                            ['lbfgs',
                                             'liblinear',
                                             'newton-cg',
                                             'newton-cholesky',
                                             'sag',
                                             'saga']),
        'C': trial.suggest_float('C', 1, 100, log=True)
        }

    polynomial_degree = trial.suggest_int('degree', 1, 5)

    data_dict = preprocess_data(raw_train_df,
                                target_col='Exited',
                                scaler_numeric=True,
                                polynomial_features=True,
                                polynomial_degree=polynomial_degree)

    model = LogisticRegression(**params)
    auc_roc = cross_val_score(model,
                              data_dict['train_X'],
                              data_dict['train_y'],
                              scoring='roc_auc',
                              cv=3)

    return auc_roc.mean()


study = optuna.create_study(
    study_name='LogisticRegression_optimizer',
    direction='maximize'
)

optuna.logging.disable_default_handler()
study.optimize(objective, n_trials=50)

study.best_value, study.best_params

[I 2025-02-18 21:18:28,225] A new study created in memory with name: LogisticRegression_optimizer
d:\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\anaconda3\envs

CPU times: total: 39min 14s
Wall time: 25min 36s


(0.9319432426777059,
 {'solver': 'newton-cg', 'C': 4.707826132868454, 'degree': 5})

In [33]:
%%time
data_dict_poly_4 = preprocess_data(raw_train_df,
                                   target_col='Exited',
                                   scaler_numeric=True,
                                   polynomial_features=True,
                                   polynomial_degree=study.best_params['degree'])

model = LogisticRegression(class_weight='balanced',
                           max_iter=500,
                           random_state=24,
                           solver=study.best_params['solver'],
                           C=study.best_params['C'])

model.fit(data_dict_poly_4['train_X'], data_dict_poly_4['train_y'])

train_auroc, val_auroc = compute_auroc(model, data_dict_poly_4)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")

AUROC on Train: 0.941
AUROC on Validation: 0.928
CPU times: total: 33.3 s
Wall time: 9.2 s


This is the best result! Let's create submit to Kaggle.

In [31]:
test_df = pd.read_csv(data_dir + 'test.csv')

test_inputs = preprocess_new_data(
    test_df,
    data_dict_poly_4['input_cols'],
    data_dict_poly_4['numeric_cols'],
    data_dict_poly_4['categorical_cols'],
    data_dict_poly_4['poly_cols'],
    data_dict_poly_4['encoder'],
    data_dict_poly_4['scaler'],
    data_dict_poly_4['poly_transformer']
)

test_preds_proba = model.predict_proba(test_inputs)[:, 1]

submission['Exited'] = test_preds_proba

submission.to_csv(data_dir + 'submission_log_reg_optuna.csv',
                  index=False)

Result on Kaggle: `0.93325`